# Check GPU availability
Check GPU drivers and CUDA availability for training.

In [ ]:
# Check GPU status
!nvidia-smi

# Install required packages
Install required packages for downloading dataset and training YOLOv11 model.

In [ ]:
# Install required packages
print("Installing required packages...")
print("="*70)

# Install packages
%pip install -q ultralytics roboflow openvino nncf

print("\n" + "="*70)
print("✓ Packages installed successfully!")
print("  The OpenVINO export will work correctly with the installed versions.")

In [ ]:
# Import necessary libraries
from google.colab import userdata
import os

# Roboflow configuration
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
ROBOFLOW_WORKSPACE = userdata.get('ROBOFLOW_WORKSPACE')
ROBOFLOW_PROJECT = userdata.get('ROBOFLOW_PROJECT')
ROBOFLOW_VERSION = userdata.get('ROBOFLOW_VERSION')

# Model configuration
MODEL_NAME = "yolo11n-seg-version-1-0-0"
MODEL_PATH = f"/content/{MODEL_NAME}.pt"

print("Configuration loaded successfully!")
print(f"Model: {MODEL_NAME}")
print(f"Model path: {MODEL_PATH}")

# Download dataset from Roboflow
Download dataset from Roboflow for YOLOv11 training.

In [ ]:
# Download dataset
from roboflow import Roboflow

print("Downloading dataset from Roboflow...")
print(f"Workspace: {ROBOFLOW_WORKSPACE}")
print(f"Project: {ROBOFLOW_PROJECT}")
print(f"Version: {ROBOFLOW_VERSION}")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(int(ROBOFLOW_VERSION))
dataset = version.download("yolov11")

print("Dataset downloaded successfully!")

# Export OpenVINO model
Export the trained YOLOv11 model to OpenVINO format for deployment.

In [ ]:
from ultralytics import YOLO
import os

# Define model and dataset paths
DATASET_PATH = f"{dataset.location}/data.yaml"

print(f"Model path: {MODEL_PATH}")
print(f"Dataset path: {DATASET_PATH}")
print("="*70)

# Load the YOLO11n-seg PyTorch model
print("\nLoading PyTorch model...")
model = YOLO(MODEL_PATH)
print("✓ Model loaded successfully!")

# Export the model to OpenVINO format with INT8 Quantization
print("\n" + "="*70)
print("EXPORTING TO OPENVINO FORMAT WITH INT8 QUANTIZATION")
print("="*70)
print("\nExport parameters:")
print(f"  - Format: OpenVINO")
print(f"  - Image size: 640x640")
print(f"  - Quantization: INT8 (8-bit integer)")
print(f"  - Calibration data: {DATASET_PATH}")
print(f"  - Calibration fraction: 0.5 (50% of dataset)")
print("\nStarting export... (this may take several minutes)")

# Export the model with INT8 quantization
# The export creates 'yolo11n-seg-version-1-0-0_int8_openvino_model/' directory
openvino_model_path = model.export(
    format="openvino",   # Export format
    imgsz=640,           # Input image size
    int8=True,           # Enable INT8 quantization
    data=DATASET_PATH,   # Dataset for calibration (required for INT8)
    fraction=0.5,        # Fraction of calibration data to use (0.0-1.0)
    half=False,          # Disable FP16 since we're using INT8
)

print("\n" + "="*70)
print("✓ Export completed successfully!")
print(f"✓ OpenVINO model saved to: {openvino_model_path}")
print("="*70)

# List exported files
export_dir = openvino_model_path
if os.path.isdir(export_dir):
    print("\nExported files:")
    for f in os.listdir(export_dir):
        file_path = os.path.join(export_dir, f)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"  - {f}: {size_mb:.2f} MB")

# Model Validation (Segmentation)

Evaluate segmentation model performance on validation set.
Metrics include both bounding box (box) and mask (seg) metrics.

In [ ]:
# Load the exported OpenVINO INT8 model for validation
print("Loading OpenVINO INT8 model for validation...")
openvino_model = YOLO(openvino_model_path)
print("✓ OpenVINO model loaded successfully!")

# Validate the model on validation set
print("\n" + "="*70)
print("RUNNING VALIDATION ON VALIDATION SET")
print("="*70)

metrics = openvino_model.val(data=DATASET_PATH, split='val')

# Print detailed metrics for both Box and Mask
print("\n" + "="*70)
print("SEGMENTATION VALIDATION METRICS (OpenVINO INT8)")
print("="*70)

# Bounding Box Metrics
print("\nBOUNDING BOX METRICS:")
print("-"*40)
print(f"mAP50-95 (Box): {metrics.box.map:.4f}")      # mAP at IoU 0.5:0.95
print(f"mAP50 (Box):    {metrics.box.map50:.4f}")    # mAP at IoU 0.5
print(f"mAP75 (Box):    {metrics.box.map75:.4f}")    # mAP at IoU 0.75
print(f"Precision (Box): {metrics.box.mp:.4f}")      # Mean precision
print(f"Recall (Box):    {metrics.box.mr:.4f}")      # Mean recall

# Segmentation Mask Metrics
print("\nSEGMENTATION MASK METRICS:")
print("-"*40)
print(f"mAP50-95 (Mask): {metrics.seg.map:.4f}")     # mAP at IoU 0.5:0.95 for masks
print(f"mAP50 (Mask):    {metrics.seg.map50:.4f}")   # mAP at IoU 0.5 for masks
print(f"mAP75 (Mask):    {metrics.seg.map75:.4f}")   # mAP at IoU 0.75 for masks
print(f"Precision (Mask): {metrics.seg.mp:.4f}")     # Mean precision for masks
print(f"Recall (Mask):    {metrics.seg.mr:.4f}")     # Mean recall for masks

print("\n" + "="*70)

# Print per-class metrics if available
if hasattr(metrics.box, 'maps') and hasattr(metrics.seg, 'maps'):
    print("\nPer-class Metrics (Validation Set):")
    print(f"{'Class':<15} {'Box mAP50-95':<15} {'Mask mAP50-95':<15}")
    print("-"*45)
    for i, (box_map, seg_map) in enumerate(zip(metrics.box.maps, metrics.seg.maps)):
        print(f"Class {i:<9} {box_map:<15.4f} {seg_map:<15.4f}")

# Store validation metrics for comparison
val_metrics = {
    'box_map': metrics.box.map,
    'box_map50': metrics.box.map50,
    'seg_map': metrics.seg.map,
    'seg_map50': metrics.seg.map50
}

print("\n✓ Validation completed!")

# Model Testing (Segmentation) - Test Dataset

Evaluate segmentation model performance on **test dataset**.
This provides an unbiased evaluation of the final model on unseen data.
Metrics include both bounding box (box) and mask (seg) metrics.

In [ ]:
# Evaluate on TEST dataset
print("="*70)
print("RUNNING EVALUATION ON TEST SET")
print("="*70)
print("This evaluates the OpenVINO INT8 model on unseen test data.\n")

test_metrics = openvino_model.val(data=DATASET_PATH, split='test')

# Print detailed metrics for both Box and Mask
print("\n" + "="*70)
print("SEGMENTATION TEST METRICS (OpenVINO INT8 - TEST DATASET)")
print("="*70)

# Bounding Box Metrics
print("\nBOUNDING BOX METRICS:")
print("-"*40)
print(f"mAP50-95 (Box): {test_metrics.box.map:.4f}")      # mAP at IoU 0.5:0.95
print(f"mAP50 (Box):    {test_metrics.box.map50:.4f}")    # mAP at IoU 0.5
print(f"mAP75 (Box):    {test_metrics.box.map75:.4f}")    # mAP at IoU 0.75
print(f"Precision (Box): {test_metrics.box.mp:.4f}")      # Mean precision
print(f"Recall (Box):    {test_metrics.box.mr:.4f}")      # Mean recall

# Segmentation Mask Metrics
print("\nSEGMENTATION MASK METRICS:")
print("-"*40)
print(f"mAP50-95 (Mask): {test_metrics.seg.map:.4f}")     # mAP at IoU 0.5:0.95 for masks
print(f"mAP50 (Mask):    {test_metrics.seg.map50:.4f}")   # mAP at IoU 0.5 for masks
print(f"mAP75 (Mask):    {test_metrics.seg.map75:.4f}")   # mAP at IoU 0.75 for masks
print(f"Precision (Mask): {test_metrics.seg.mp:.4f}")     # Mean precision for masks
print(f"Recall (Mask):    {test_metrics.seg.mr:.4f}")     # Mean recall for masks

print("\n" + "="*70)

# Print per-class metrics if available
if hasattr(test_metrics.box, 'maps') and hasattr(test_metrics.seg, 'maps'):
    print("\nPer-class Metrics (Test Set):")
    print(f"{'Class':<15} {'Box mAP50-95':<15} {'Mask mAP50-95':<15}")
    print("-"*45)
    for i, (box_map, seg_map) in enumerate(zip(test_metrics.box.maps, test_metrics.seg.maps)):
        print(f"Class {i:<9} {box_map:<15.4f} {seg_map:<15.4f}")

# Compare Validation vs Test metrics
print("\n" + "="*70)
print("COMPARISON: VALIDATION vs TEST (OpenVINO INT8)")
print("="*70)
print(f"{'Metric':<25} {'Validation':<15} {'Test':<15} {'Difference':<15}")
print("-"*70)

# Box metrics comparison
diff_box = test_metrics.box.map - val_metrics['box_map']
print(f"{'Box mAP50-95':<25} {val_metrics['box_map']:<15.4f} {test_metrics.box.map:<15.4f} {diff_box:+.4f}")

diff_box50 = test_metrics.box.map50 - val_metrics['box_map50']
print(f"{'Box mAP50':<25} {val_metrics['box_map50']:<15.4f} {test_metrics.box.map50:<15.4f} {diff_box50:+.4f}")

# Mask metrics comparison
diff_seg = test_metrics.seg.map - val_metrics['seg_map']
print(f"{'Mask mAP50-95':<25} {val_metrics['seg_map']:<15.4f} {test_metrics.seg.map:<15.4f} {diff_seg:+.4f}")

diff_seg50 = test_metrics.seg.map50 - val_metrics['seg_map50']
print(f"{'Mask mAP50':<25} {val_metrics['seg_map50']:<15.4f} {test_metrics.seg.map50:<15.4f} {diff_seg50:+.4f}")

print("-"*70)

# Analysis
if abs(diff_box) < 0.05 and abs(diff_seg) < 0.05:
    print("✓ Model generalizes well - small difference between validation and test.")
elif diff_box < -0.1 or diff_seg < -0.1:
    print("⚠ Model may be overfitting - significant drop in test performance.")
else:
    print("✓ Test performance is consistent with validation.")

print("\n✓ Test evaluation completed!")

# Save Models to Google Drive

Save all model files to Google Drive for persistent storage.
Includes:
- Original PyTorch model (.pt)
- OpenVINO INT8 quantized model (.xml & .bin)

In [ ]:
import shutil
import os
from datetime import datetime

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_base_dir = f"/content/drive/MyDrive/YOLO_Models/{MODEL_NAME}"
output_dir = f"{output_base_dir}/{timestamp}"

os.makedirs(output_dir, exist_ok=True)
print(f"Output directory: {output_dir}")
print("="*70)

# 1. Copy original PyTorch model
print("\n1. Saving PyTorch model...")
pt_dest = os.path.join(output_dir, f"{MODEL_NAME}.pt")
shutil.copy2(MODEL_PATH, pt_dest)
print(f"   ✓ Saved: {pt_dest}")

# 2. Copy OpenVINO INT8 model directory
print("\n2. Saving OpenVINO INT8 model...")
openvino_dest = os.path.join(output_dir, f"{MODEL_NAME}_int8_openvino_model")
if os.path.isdir(openvino_model_path):
    shutil.copytree(openvino_model_path, openvino_dest)
    print(f"   ✓ Saved: {openvino_dest}")

    # List OpenVINO model files
    print("   OpenVINO model contents:")
    for f in os.listdir(openvino_dest):
        file_path = os.path.join(openvino_dest, f)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"      - {f}: {size_mb:.2f} MB")

# 3. Save validation and test metrics to a text file
print("\n3. Saving evaluation metrics...")
metrics_file = os.path.join(output_dir, "evaluation_metrics.txt")

with open(metrics_file, 'w') as f:
    f.write("="*70 + "\n")
    f.write(f"YOLO11n-seg OpenVINO INT8 Model Evaluation Report\n")
    f.write(f"Model: {MODEL_NAME}\n")
    f.write(f"Export Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*70 + "\n\n")

    f.write("VALIDATION SET METRICS:\n")
    f.write("-"*40 + "\n")
    f.write(f"Box mAP50-95:  {val_metrics['box_map']:.4f}\n")
    f.write(f"Box mAP50:     {val_metrics['box_map50']:.4f}\n")
    f.write(f"Mask mAP50-95: {val_metrics['seg_map']:.4f}\n")
    f.write(f"Mask mAP50:    {val_metrics['seg_map50']:.4f}\n\n")

    f.write("TEST SET METRICS:\n")
    f.write("-"*40 + "\n")
    f.write(f"Box mAP50-95:  {test_metrics.box.map:.4f}\n")
    f.write(f"Box mAP50:     {test_metrics.box.map50:.4f}\n")
    f.write(f"Mask mAP50-95: {test_metrics.seg.map:.4f}\n")
    f.write(f"Mask mAP50:    {test_metrics.seg.map50:.4f}\n\n")

    f.write("EXPORT CONFIGURATION:\n")
    f.write("-"*40 + "\n")
    f.write(f"Format: OpenVINO\n")
    f.write(f"Quantization: INT8\n")
    f.write(f"Image size: 640x640\n")
    f.write(f"Calibration dataset: {DATASET_PATH}\n")
    f.write(f"Calibration fraction: 0.5\n")

print(f"   ✓ Saved: {metrics_file}")

# 4. Create a latest symlink for easy access
print("\n4. Creating 'latest' reference...")
latest_link = os.path.join(output_base_dir, "latest")
latest_file = os.path.join(output_base_dir, "latest.txt")
with open(latest_file, 'w') as f:
    f.write(f"{timestamp}\n")
print(f"   ✓ Latest version recorded: {timestamp}")

# Summary
print("\n" + "="*70)
print("SAVE SUMMARY")
print("="*70)
print(f"\nAll files saved to Google Drive:")
print(f"📁 {output_dir}/")
print(f"   ├── {MODEL_NAME}.pt")
print(f"   ├── {MODEL_NAME}_int8_openvino_model/")
print(f"   │   ├── *.xml (model architecture)")
print(f"   │   └── *.bin (model weights)")
print(f"   └── evaluation_metrics.txt")
print("\n✓ All models saved successfully to Google Drive!")

# Test Inference with OpenVINO Model

Run a sample inference to verify the exported OpenVINO INT8 model works correctly.

In [ ]:
# Test inference with the exported OpenVINO INT8 model
import glob
from PIL import Image
import matplotlib.pyplot as plt

print("="*70)
print("TESTING OPENVINO INT8 MODEL INFERENCE")
print("="*70)

# Find a sample image from the dataset
sample_images = glob.glob(f"{dataset.location}/test/images/*.*")
if not sample_images:
    sample_images = glob.glob(f"{dataset.location}/valid/images/*.*")

if sample_images:
    test_image = sample_images[0]
    print(f"\nTest image: {test_image}")

    # Run inference with OpenVINO model
    print("\nRunning inference with OpenVINO INT8 model...")
    results = openvino_model.predict(
        source=test_image,
        conf=0.5,
        save=True,
        project="/content/inference_results",
        name="openvino_test"
    )

    # Display results
    print(f"\nDetection results:")
    print(f"  - Objects detected: {len(results[0].boxes)}")

    if len(results[0].boxes) > 0:
        print(f"  - Classes: {results[0].boxes.cls.tolist()}")
        print(f"  - Confidence scores: {[f'{c:.2f}' for c in results[0].boxes.conf.tolist()]}")

    # Show the result image
    result_img = results[0].plot()
    plt.figure(figsize=(12, 8))
    plt.imshow(result_img[..., ::-1])  # BGR to RGB
    plt.title("OpenVINO INT8 Model Inference Result")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    print("\n✓ Inference test completed successfully!")
else:
    print("⚠ No sample images found for testing.")